In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from typing import List
np.set_printoptions(threshold=np.inf)  
from tqdm import tqdm
from evaluate_sol import *
from fragment import *
from side import *
from sides_comparison import *
from utils import *
import global_values
from groups import *
from merge_groups_best_candidate import *
from puzzle import *
global_values.GRAD_SCORING = True
global_values.NN_SCORING = False

global_values.ROTATING_PIECES = True

# global_values.IMAGE_TH = 0.08
# global_values.GROUP_TH = 0.2

# global_values.PRUDENT_X_BUDDY_TH_IMAGE = 0.70




In [ ]:
def create_puzzle(f, image_path, output_folder):
    solution_path = f"solution/valid_comparisons_{f}_{global_values.COL_NR}_{global_values.ROW_NR}.txt"

    fragments, tile_h, tile_w , gt_grid, gt_rotations = divide_image(image_path, output_folder, solution_path, global_values.COL_NR, global_values.ROW_NR)
    global_values.TILE_H = global_values.TILE_W = max(tile_h, tile_w)
    gt_comparisons = read_valid_comparisons(solution_path)

    return fragments, gt_grid, gt_rotations, gt_comparisons



def initialise_puzzle_solver(fragments):
    print("create sides_comp")
    sides_comparisons = create_sides_comparisons(fragments)

    print("buddy score")
    sides_comparisons = calculate_buddy_score(fragments, sides_comparisons)

    print("sort sides comp")
    sorted_sides_comparisons = sort_sides_comparisons(sides_comparisons)

    n = len(fragments)
    global_values.SYMMETRIC_COMPARISONS = [[[[None for _ in range(4)] for _ in range(4)] for _ in range(n)] for _ in range(n)]

    print("copy symmetric")
    for c in tqdm(sides_comparisons):
        global_values.SYMMETRIC_COMPARISONS[c.side1.fragment_idx][c.side2.fragment_idx][c.side1.side_idx][c.side2.side_idx] = c

        symmetric_comp = SidesComparison(fragments, c.side2, c.side1, c.score)
        symmetric_comp.buddy_score = c.buddy_score
        symmetric_comp.prudent_score = c.prudent_score

        global_values.SYMMETRIC_COMPARISONS[c.side2.fragment_idx][c.side1.fragment_idx][c.side2.side_idx][c.side1.side_idx] = symmetric_comp

    groups = []
    fragment_idx_to_group_idx = [None for _ in fragments]

    for fr_idx in range(len(fragments)):
        new_group = Group(fr_idx)
        groups.append(new_group)
        fragment_idx_to_group_idx[fr_idx] = fr_idx

    return sides_comparisons, sorted_sides_comparisons, groups, fragment_idx_to_group_idx



def solve(sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups, gt_comparisons, gt_grid, f, give_prints = False):
    output_dir = f"solved_puzzles_{global_values.COL_NR * global_values.ROW_NR}"
    red_sp_1 = None
    red_sp_1_false = None
    red_sp_2 = None
    red_sp_2_false = None
    voting = None
    voting_false = None
    umplere_gol = None
    umplere_gol_false = None
    reparare = None
    reparare_false = None

    # etapa 1
    groups, fragments, fragment_idx_to_group_idx = merge_where_obvious(
        prudent_x_buddy_one_image_passing, no_mean_score, 0.6, 0.015,
        sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups)

    if give_prints:
        print("Reducerea spatiului - Prudent * Buddy score  0.6")
        fig = show_all_groups(groups, fragments, fragment_idx_to_group_idx, 0)
        red_sp_1, red_sp_1_false = get_matches_accuracy(gt_comparisons, groups, fragments)

        # if fig is not None:
        #     save_dir = os.path.join(output_dir,f)
        #     os.makedirs(save_dir, exist_ok=True)
        #     save_path = os.path.join(save_dir, f"Red_sp_1_correct_{red_sp_1:.2f}_false_{red_sp_1_false:.2f}.png")
        #     fig.savefig(save_path)
        #     plt.close(fig)
        #     print(f"Saved plot: {save_path}")

    # etapa 2
    if len(groups) > 1:
        groups, fragments, fragment_idx_to_group_idx = merge_where_obvious(
            no_image_condition, mean_clasic_score, 100, 0.02,
            sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups)

        if give_prints:
            print("Reducerea spatiului - Clasic mean score 0.02")
            fig = show_all_groups(groups, fragments, fragment_idx_to_group_idx, 0)
            red_sp_2, red_sp_2_false = get_matches_accuracy(gt_comparisons, groups, fragments)

            # if fig is not None:
            #     save_dir = os.path.join(output_dir, f)
            #     os.makedirs(save_dir, exist_ok=True)
            #     save_path = os.path.join(save_dir, f"Red_sp_2_correct_{red_sp_2:.2f}_false_{red_sp_2_false:.2f}.png")
            #     fig.savefig(save_path)
            #     plt.close(fig)
            #     print(f"Saved plot: {save_path}")

    # voting
    if len(groups) > 1:
        groups_red, fragments_red, fragment_idx_to_group_idx_red = vote_and_solve(
            groups, fragments, fragment_idx_to_group_idx,
            clasic_score_one_image_passing, mean_clasic_score, 100, 0.02)

        if give_prints:
            print("Voting - clasic mean 0.02")
            fig = show_all_groups(groups, fragments, fragment_idx_to_group_idx, 0)
            voting, voting_false = get_matches_accuracy(gt_comparisons, groups, fragments)

            # if fig is not None:
            #     save_dir = os.path.join(output_dir, f)
            #     os.makedirs(save_dir, exist_ok=True)
            #     save_path = os.path.join(save_dir, f"Voting_correct_{voting:.2f}_false_{voting_false:.2f}.png")
            #     fig.savefig(save_path)
            #     plt.close(fig)
            #     print(f"Saved plot: {save_path}")

    # completarea golurilor
    if len(groups) > 1:
        groups, fragments, fragment_idx_to_group_idx = solve_groups_safe(groups_red, fragments_red, fragment_idx_to_group_idx_red)

        if give_prints:
            print("Completarea golurilor")
            fig = show_all_groups(groups, fragments, fragment_idx_to_group_idx, 0)
            umplere_gol, umplere_gol_false = get_matches_accuracy(gt_comparisons, groups, fragments)

            # if fig is not None:
            #     save_dir = os.path.join(output_dir, f)
            #     os.makedirs(save_dir, exist_ok=True)
            #     save_path = os.path.join(save_dir, f"Umplere_gol_correct_{umplere_gol:.2f}_false_{umplere_gol_false:.2f}.png")
            #     fig.savefig(save_path)
            #     plt.close(fig)
            #     print(f"Saved plot: {save_path}")

    # reparare
    if len(groups) > 1:
        groups, fragment_idx_to_group_idx, big_group_idx = undo_small_groups(groups, fragment_idx_to_group_idx)
        groups, fragments, fragment_idx_to_group_idx = solve_groups_safe_reparing(groups, fragments, fragment_idx_to_group_idx, big_group_idx)

        if give_prints:
            print("Reparing")
            fig = show_all_groups(groups, fragments, fragment_idx_to_group_idx, 0)
            reparare, reparare_false = get_matches_accuracy(gt_comparisons, groups, fragments)
            
            # if fig is not None:
            #     save_dir = os.path.join(output_dir, f)
            #     os.makedirs(save_dir, exist_ok=True)
            #     save_path = os.path.join(save_dir, f"Reparare_correct_{reparare:.2f}_false_{reparare_false:.2f}.png")
            #     fig.savefig(save_path)
            #     plt.close(fig)
            #     print(f"Saved plot: {save_path}")

    distance = solution_distace(groups[0].grid, gt_grid)
    if give_prints:
        print(f"Distance: {distance}")

    return red_sp_1, red_sp_1_false, red_sp_2, red_sp_2_false, voting, voting_false, umplere_gol, umplere_gol_false, reparare, reparare_false, distance


In [ ]:

global_values.COL_NR = 2
global_values.ROW_NR = 2
import csv

results_file = f"results_{global_values.COL_NR}.csv"
with open(results_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["filename", "red_sp_1", "red_sp_1_false", "red_sp_2", "red_sp_2_false", "voting", "voting_false", "umplere_gol", "umplere_gol_false", "reparare", "reparare_false", "final_distance"])

image_dir = "all_images"
files = ["0419.png"]
# files = os.listdir(image_dir)
for f in files:
    print(f"\nProcessing {f}\n")
    f_base = os.path.splitext(f)[0]
    image_path = os.path.join(image_dir, f)
    output_folder = "fragments"

    fragments, gt_grid, gt_rotations, gt_comparisons = create_puzzle(f, image_path, output_folder)

    sides_comparisons, sorted_sides_comparisons, groups, fragment_idx_to_group_idx = initialise_puzzle_solver(fragments)

    red_sp_1, red_sp_1_false, red_sp_2, red_sp_2_false, voting, voting_false, umplere_gol, umplere_gol_false, reparare, reparare_false, distance = solve(sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups, gt_comparisons, gt_grid,f_base, 1)

    with open(results_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f, red_sp_1, red_sp_2, voting, umplere_gol, reparare, distance])



In [ ]:

global_values.COL_NR = 20
global_values.ROW_NR = 20
import csv

results_file = f"results_{global_values.COL_NR}.csv"

image_dir = "plotting_images"
# files = ["0006.png"]
files = os.listdir(image_dir)
for f in files:
    print(f"\nProcessing {f}\n")
    f_base = os.path.splitext(f)[0]
    image_path = os.path.join(image_dir, f)
    output_folder = "fragments"

    fragments, gt_grid, gt_rotations, gt_comparisons = create_puzzle(f, image_path, output_folder)

    sides_comparisons, sorted_sides_comparisons, groups, fragment_idx_to_group_idx = initialise_puzzle_solver(fragments)

    red_sp_1, red_sp_1_false, red_sp_2, red_sp_2_false, voting, voting_false, umplere_gol, umplere_gol_false, reparare, reparare_false, distance = solve(sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups, gt_comparisons, gt_grid,f_base, 1)

    with open(results_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f, red_sp_1, red_sp_2, voting, umplere_gol, reparare, distance])



In [ ]:

global_values.COL_NR = 2
global_values.ROW_NR = 2
import csv

results_file = f"results_{global_values.COL_NR}.csv"

image_dir = "all_images"
# files = ["0006.png"]
files = os.listdir(image_dir)
for f in files:
    print(f"\nProcessing {f}\n")
    f_base = os.path.splitext(f)[0]
    image_path = os.path.join(image_dir, f)
    output_folder = "fragments"

    fragments, gt_grid, gt_rotations, gt_comparisons = create_puzzle(f, image_path, output_folder)

    sides_comparisons, sorted_sides_comparisons, groups, fragment_idx_to_group_idx = initialise_puzzle_solver(fragments)

    red_sp_1, red_sp_1_false, red_sp_2, red_sp_2_false, voting, voting_false, umplere_gol, umplere_gol_false, reparare, reparare_false, distance = solve(sorted_sides_comparisons, fragment_idx_to_group_idx, fragments, groups, gt_comparisons, gt_grid,f_base, 1)

    with open(results_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f, red_sp_1, red_sp_2, voting, umplere_gol, reparare, distance])



In [ ]:
# introducere problema imagini input output

# care e algolritmul (greedy)
# functia de potrivire
# siameza

# cum merge pe low level, simaza pe diferiti nr pixeli
# timp

# 9 iunie licenta
